In [ ]:
# Install packages
!pip install h5py

In [21]:
import os
import subprocess
import h5py
import h5py
import os
import json

In [ ]:
# get all files from a directory
def get_all_files(directory):
    return [f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))]

In [ ]:
# convert .h5 to .vrt files

def h5_to_vrt(filename):
    input_name = filename
    output_name = filename.replace('.h5', '.vrt')


    cmd = [
        'gdal_translate',
        '-of', 'VRT',                
        '-ot', 'Byte',               
        '-a_nodata', '-28,-29,-30,-31,-32,-33,-34,-35,-36,-37',  
        '-scale', '-28', '36', '0', '255',  
        input_name,                  
        output_name,               
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"VRT file '{output_name}' created successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error while creating VRT file: {e}")

In [27]:
# convert .vrt to xyz tiles

def vrt_to_tiles(filename):
    input_name = filename
    output_name = filename.replace('.vrt', '_tiles')

    cmd = [
        'gdal2tiles.py',
        '--xyz',
        '-z', '6-10',               
        '-w', 'none',               
        input_name,                 
        output_name,                
    ]

    try:
        subprocess.run(cmd, check=True)
        print(f"Tiles directory '{output_name}' created successfully.")
    except subprocess.CalledProcessError as e:
        print(f"Error while creating tiles directory: {e}")

In [ ]:
for file in get_all_files('./tiles/'):
    print(file)
    h5_to_vrt('./tiles/' + file)


CZC221240020VL.801.h5
Input file size is 710, 640
VRT file './tiles/CZC221240020VL.801.vrt' created successfully.
CZC221240020VL.801.vrt
Input file size is 710, 640
Usage: gdal_translate [--help-general] [--long-usage]
       [-ot {Byte/Int16/UInt16/UInt32/Int32/Float32/Float64/
             CInt16/CInt32/CFloat32/CFloat64}] [-strict]
       [-if format]* [-of format]
       [-b band] [-mask band] [-expand {gray|rgb|rgba}]
       [-outsize xsize[%]|0 ysize[%]|0] [-tr xres yres]
       [-r {nearest,bilinear,cubic,cubicspline,lanczos,average,mode}]
       [-unscale] [-scale[_bn] [src_min src_max [dst_min dst_max]]]* [-exponent[_bn] exp_val]*
       [-srcwin xoff yoff xsize ysize] [-epo] [-eco]
       [-projwin ulx uly lrx lry] [-projwin_srs srs_def]
       [-a_srs srs_def] [-a_coord_epoch epoch]
       [-a_ullr ulx uly lrx lry] [-a_nodata value]
       [-a_scale value] [-a_offset value]
       [-nogcp] [-gcp pixel line easting northing [elevation]]*
       |-colorinterp{_bn} {red|green|b

Warning 1: for band 1, nodata value has been clamped to 0, the original value being out of range.
ERROR 1: Source and destination datasets must be different.


In [29]:
# run the conversion for all files in the directory

for file in get_all_files('./CZC/'):
    if '.vrt' in file:
        vrt_to_tiles('./CZC/' + file.replace('.h5', '.vrt'))

Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
Tiles directory './CZC/CZC221241225VL.801_tiles' created successfully.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
Tiles directory './CZC/CZC221241855VL.801_tiles' created successfully.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
Tiles directory './CZC/CZC221240135VL.801_tiles' created successfully.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overview Tiles:
0...10...20...30...40...50...60...70...80...90...100
Tiles directory './CZC/CZC221242220VL.801_tiles' created successfully.
Generating Base Tiles:
0...10...20...30...40...50...60...70...80...90...100
Generating Overv

In [51]:
# get the date and time from the hdf5 file

def get_custom_iso_datetime_from_hdf5(file_path):
    try:
        with h5py.File(file_path, 'r') as hdf5_file:
            if 'what' in hdf5_file:
                what_group = hdf5_file['what']
                date_bytes = what_group.attrs.get('date', None)
                time_bytes = what_group.attrs.get('time', None)
                
                if date_bytes and time_bytes:
                    # Decode bytes to UTF-8 and format date and time with hyphens, colons, milliseconds, and "Z"
                    date = date_bytes.decode('utf-8')
                    time = time_bytes.decode('utf-8')
                    iso_datetime = f"{date[:4]}-{date[4:6]}-{date[6:8]}T{time[:2]}:{time[2:4]}:{time[4:6]}.000Z"
                    return iso_datetime
                else:
                    return None
            else:
                return None
    except Exception as e:
        print(f"Error while reading HDF5 file: {e}")
        return None

# Example usage:
file_path = 'CZC221240020VL.801.h5'
iso_datetime = get_custom_iso_datetime_from_hdf5(file_path)
if iso_datetime:
    print(f'Custom ISO Date and Time: {iso_datetime}')
else:
    print('Unable to retrieve Custom ISO Date and Time from the HDF5 file.')


Custom ISO Date and Time: 2022-05-04T00:20:00.000Z


In [53]:
# create a json file with the id and start date of the tiles
directory = './CZC/'

objects_list = []

for file in os.listdir(directory):
    if file.endswith('.h5'):
        id = os.path.splitext(file)[0]
        start_date = get_custom_iso_datetime_from_hdf5(os.path.join(directory, file))

        if start_date:
            object_data = {'id': id + '_tiles', 'startDate': start_date}
            objects_list.append(object_data)

json_array = json.dumps(objects_list, indent=4)

[
    {
        "id": "CZC221241345VL.801",
        "startDate": "2022-05-04T13:45:00.000Z"
    },
    {
        "id": "CZC221241155VL.801",
        "startDate": "2022-05-04T11:55:00.000Z"
    },
    {
        "id": "CZC221241535VL.801",
        "startDate": "2022-05-04T15:35:00.000Z"
    },
    {
        "id": "CZC221241510VL.801",
        "startDate": "2022-05-04T15:10:00.000Z"
    },
    {
        "id": "CZC221240620VL.801",
        "startDate": "2022-05-04T06:20:00.000Z"
    },
    {
        "id": "CZC221241335VL.801",
        "startDate": "2022-05-04T13:35:00.000Z"
    },
    {
        "id": "CZC221241005VL.801",
        "startDate": "2022-05-04T10:05:00.000Z"
    },
    {
        "id": "CZC221240710VL.801",
        "startDate": "2022-05-04T07:10:00.000Z"
    },
    {
        "id": "CZC221241640VL.801",
        "startDate": "2022-05-04T16:40:00.000Z"
    },
    {
        "id": "CZC221240630VL.801",
        "startDate": "2022-05-04T06:30:00.000Z"
    },
    {
        "id": "CZC2212

In [ ]:
# Write the JSON array to a file, this can be used to map the start date to the id of the tiles

output_file = 'layers.json'

with open(output_file, 'w') as file:
    file.write(json_array)